In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommender: Movielens1M, load_dataset, ratio_split, ImplicitMF, evaluate_u2i, PrecisionAtK, RecallAtK, NDCG, fit!, predict_u2i, make_u2i_dataset

In [2]:
ml1M = Movielens1M()
download(ml1M)
rating, user, movie = load_dataset(ml1M);

In [3]:
rating = rating |> TableOperations.filter(x->Tables.getcolumn(x, :rating) >= 4)

TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens1m/ratings.dat"):
Size: 1000209 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64)

In [4]:
movie2title = Dict()
for row in Tables.rows(movie)
    movie2title[row[:movieid]] = row[:title]
end
rating = rating |> TableOperations.transform(Dict(:movieid=>x->movie2title[x]))

TableOperations.Transforms{false, TableOperations.Filter{var"#1#2", CSV.File{false}}, Dict{Symbol, var"#3#4"}}(TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens1m/ratings.dat"):
Size: 1000209 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64), Dict(:movieid => var"#3#4"()))

In [5]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))

(368180, 92045, 115056)

In [6]:
prec10 = PrecisionAtK(10)
recall10 = RecallAtK(10)
ndcg10 = NDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommender.AbstractMetric}:
 PrecisionAtK(10)
 RecallAtK(10)
 NDCG(10)

In [7]:
model = ImplicitMF(128, true, 0.005)

ImplicitMF(128, true, 0.005, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [8]:
evaluate_u2i(model, train_valid_table, test_table, metrics, 10, col_item=:movieid, n_epochs=256, n_negatives=8, learning_rate=0.002, drop_history=true)

┌ Info: epoch=1: loss=Inf, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=2: loss=0.36246173237387935, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=3: loss=0.33172135607075137, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=4: loss=0.31273707803755907, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=5: loss=0.3001929955036619, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=6: loss=0.2911181400511353, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:142
┌ Info: epoch=7: loss=0.2838532136645056, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/im